Let’s fire up the next-generation architecture trail — starting with one of the most exciting frontiers in modern ML:

---

# 🧪 `07_lab_gnn_node_classification_with_cora.ipynb`  
### 📁 `04_advanced_architectures`  
> Build a **Graph Neural Network (GNN)** to classify nodes in the **Cora citation network**.  
You’ll see how graphs differ from images or sequences — and why **message passing** is the future for structured data.

---

## 🎯 Learning Objectives

- Understand the **core logic of GNNs** (message passing + aggregation)  
- Run **node classification** on Cora with PyTorch Geometric (PyG)  
- Visualize graph structure and predictions  
- Colab/laptop-friendly setup (CPU fallback, small graphs)

---

## 💻 Runtime Targets

| Spec                 | Setting        |
|----------------------|----------------|
| Library              | ✅ PyTorch Geometric (PyG)  
| Dataset              | ✅ Cora (1400 nodes, built-in)  
| Hardware             | ✅ Colab / CPU / GPU (T4)  
| Memory               | ✅ <1GB  
| Model                | ✅ 2-layer GCN

---

## ⚙️ Section 1: Install & Import

```python
# Install PyG (for Colab)
!pip install torch_geometric torch-scatter torch-sparse -q

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import matplotlib.pyplot as plt
import seaborn as sns
```

---

## 🧠 Section 2: Load Cora Dataset

```python
dataset = Planetoid(root='./data/Cora', name='Cora')
data = dataset[0]

print(data)
# Output: Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708])
```

> 📌 2708 nodes (papers), 1433 features (words), 7 classes (topics)  
> `edge_index` defines connections — the **graph structure**

---

## 🔧 Section 3: Build GCN Model

```python
from torch.nn import Module

class GCN(Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x
```

---

## 🏃 Section 4: Train the GNN

```python
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    logits = model(data.x, data.edge_index)
    pred = logits.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        correct = pred[mask] == data.y[mask]
        accs.append(int(correct.sum()) / int(mask.sum()))
    return accs

for epoch in range(1, 201):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 20 == 0:
        print(f"Epoch {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}")
```

---

## 🎨 Section 5: Visualize Node Embeddings (t-SNE)

```python
from sklearn.manifold import TSNE

model.eval()
z = model(data.x, data.edge_index).detach().numpy()
tsne = TSNE(n_components=2)
z_2d = tsne.fit_transform(z)

plt.figure(figsize=(8, 6))
sns.scatterplot(x=z_2d[:, 0], y=z_2d[:, 1], hue=data.y.numpy(), palette='tab10', legend='full')
plt.title("Cora Node Embeddings After GNN")
plt.grid(True)
plt.show()
```

---

## ✅ Lab Wrap-Up

| Concept                       | ✅ |
|-------------------------------|----|
| Graph structure + node types | ✅ |
| GCN layers (message passing) | ✅ |
| Cora dataset usage           | ✅ |
| Train loop + eval            | ✅ |
| Visualization                | ✅ |
| Colab/laptop friendly        | ✅ |

---

## 🧠 What You Learned

- GNNs work by **sharing features with neighbors** — via the graph topology  
- Each layer = **one round of message passing**  
- GCNs can uncover **latent node clusters** even from simple graphs  
- You now understand how to **process graphs as data** 🚀

---

Next up?  
Shall we go sci-fi mode with `08_lab_memory_augmented_net_tiny_tasks.ipynb` and build a toy Neural Turing Machine that can *copy* and *recall*?